In [2]:
# Import all dependencies
import numpy as np
import pandas as pd
#In order to show all columns available
pd.set_option('display.max_columns', 200)

#Sklearn imports
from sklearn.preprocessing import LabelEncoder

#file system management
import os

#Graphing libs
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Load in ALL dataframes

apptrain = pd.read_csv('./Dataset/application_train.csv')
apptest = pd.read_csv('./Dataset/application_test.csv')
bureau = pd.read_csv('./Dataset/bureau.csv')
bureau_balance = pd.read_csv('./Dataset/bureau_balance.csv')
credit_card_balance = pd.read_csv('./Dataset/credit_card_balance.csv')
installment_payments = pd.read_csv('./Dataset/installments_payments.csv')
pos_cash_balance = pd.read_csv('./Dataset/POS_CASH_balance.csv')
previous_application = pd.read_csv('./Dataset/previous_application.csv')